## Подготовка

In [1]:
# Библиотеки
import pandas as pd

# Для форматирования вывода
BOLD = '\033[1m'
END = '\033[0m'

Считываем таблицу синонимов watset.

In [2]:
data = pd.read_table(
    'watset-mcl-mcl-synsets.tsv', names=['words'], 
    sep='\t', usecols=[2])
data['relations'] = pd.read_table(
    'watset-mcl-mcl-joint-exp-linked.tsv', names=['relations'], 
    sep='\t', usecols=[4])
data.head()

,words,relations
0,"мастерской, помпезный, искусственный, сахарный...",хороший#3:0.712557
1,"уступчивый, мирный, тише_воды_ниже_травы, жало...",хороший#3:0.551321
2,"заведомо, ни_дать_ни_взять, тика_в_тику, отож,...","да#3:0.368931, правильно#1:0.305335"
3,"поворотливый, прыткий, проныра, юркий, правомо...","хороший#3:0.363612, человек#4:0.219069"
4,"излупцевать, задать_взбучку, огреть, откатать,...",NaN


Разбиваем набор слов на список списков.

In [3]:
watset_dictionary = list()
watset_dictionary.append('')
watlink_dictionary = list()
watlink_dictionary.append('')
for i in range(0,len(data)):
    buf = data.iloc[i, :].words #words -> список слов
    watset_dictionary.append(buf.split(', '))
    
    buf = data.iloc[i, :].relations #relations -> список словарей (слово - значение)
    if type(buf) is not float:
        buf = buf.split(', ')
        buf_dict = dict()
        for j in buf:
            buf_one = j.split('#')
            buf_one[1] = buf_one[1][2:]
            buf_dict[buf_one[0]] = buf_one[1]
            watlink_dictionary.append(buf_dict)

Создадим список предлогов, чтобы их выкинуть из будущего предложения.

In [4]:
prepositions = ['в', 'без', 'до', 'из', 'к', 'на', 'по', 'о', 'от', 'перед', 'при', 'через',
                'с', 'у', 'и', 'за', 'над', 'для', 'об', 'под', 'про', 'то', 'это', 'а']

## Функции обработки

In [5]:
# Форматирование запроса в "рабочий" вид.
def formatting(sentence, word):
    """Возвращает массив слов, из которых состоит предложение за вычетом предлогов и запрашиваемого слова."""
    table = {ord(char): None for char in '!?,.;'}  # Удаляем посторонние символы
    sentence = sentence.translate(table)
    
    sentence = sentence.lower()  
    words_in_sentence = sentence.split(' ')  
    words_in_sentence.remove(word)  # Удаляем исходное слово из списка соседних по предложению слов.  
    
    words_in_sentence = [x for x in words_in_sentence if x not in prepositions] # Удаляем предлоги
    
    print(BOLD + "Соседние слова:" + END, words_in_sentence)
    return words_in_sentence

In [6]:
# Поиск номеров строк, где есть искомое слово.
def find_mentioning():
    """Возвращает словарь, ключи которого — номера строк со словом, значения везде равны нулю"""
    results = dict()
    for i in range(0,len(watset_dictionary)):
        if (word in watset_dictionary[i]):
            results[i] = 0
    print(BOLD + 'Номера строк:' + END, results.keys())
    return results

In [7]:
# Поиск в найденных строках слов, которые есть в искомом предложении.
def find_similarity(results, words_in_sentence):
    """Возвращает словарь, ключи - номера строк со словом, значения - баллы совпадения этой строки"""
    for i in results.keys():
        for close_word in words_in_sentence:
            if (close_word in watset_dictionary[i]):
                results[i] = results[i] + 1
            if (close_word in watlink_dictionary[i]):
                results[i] = results[i] + float(watlink_dictionary[i][close_word])
    print(BOLD + 'Номера строк и количество совпадений' + END, results)
    return results

In [8]:
# Находим лучшую строку и процентное соотношение.
def find_best(results):
    summary = 0
    max = 0
    for i in results.keys():
        summary = summary + results[i]
        if (results[i] > max):
            max = results[i]
            index = i

    print(BOLD + 'Синонимы:' + END)
    if (summary > 0):
        print(watset_dictionary[index])
        print(BOLD + 'Связанные слова:' + END)
        for key in watlink_dictionary[index].keys():
            print(key)
        print(BOLD + 'Совпадение по базе данных:' + END, str(max/summary * 100) + "%")
    else:
        print("not found")

## Работа с входными данными

In [9]:
sentence = (
    'Если подросток добрый, уступчивый и хороший, '
    + 'то это часто воспринимается окружающими как проявление слабости его характера, '
    + 'как неспособность чётко высказать свою позицию.')
word = 'уступчивый'

print(sentence)

Если подросток добрый, уступчивый и хороший, то это часто воспринимается окружающими как проявление слабости его характера, как неспособность чётко высказать свою позицию.


In [10]:
words_in_sentence = formatting(sentence, word)
mentions = find_mentioning()
similarity = find_similarity(mentions, words_in_sentence)
find_best(similarity)

Соседние слова: ['если', 'подросток', 'добрый', 'хороший', 'часто', 'воспринимается', 'окружающими', 'как', 'проявление', 'слабости', 'его', 'характера', 'как', 'неспособность', 'чётко', 'высказать', 'свою', 'позицию']
Номера строк: dict_keys([2, 1224, 6287])
Номера строк и количество совпадений {2: 1.551321, 1224: 0, 6287: 0}
Синонимы:
['уступчивый', 'мирный', 'тише_воды_ниже_травы', 'жалостливый', 'мешковатый', 'медлительный', 'добродушный', 'спокойный', 'милостливый', 'смиренный', 'схимник', 'доброжелательный', 'бесшумный', 'смиренник', 'благоволительный', 'добросердечный', 'безгневный', 'терпеливый', 'чуткий', 'милосердый', 'отзывчивый', 'смирный', 'гуманный', 'неразговорчивый', 'миролюбивый', 'безбурный', 'душевный', 'милостивый', 'добр', 'медленный', 'протяжный', 'человечный', 'тихий', 'безветренный', 'добродетельный', 'рахманный', 'невысказанный', 'добряк', 'благодушный', 'скромный', 'сострадательный', 'безмятежный', 'безжелчный', 'восприимчивый', 'беззвучный', 'благий', 'умолча